In [1]:
import pongGym
import random
import numpy as np
import os
env = pongGym.DoublePong()
PAD_HEIGHT = 80
HALF_PAD_HEIGHT = PAD_HEIGHT // 2

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def encode(state):
    pd1y = state[21]
    pd2y = state[23]
    st = [(max(0, state[i*4+0] + state[i*4+2] * 4), abs(state[i*4+1] + state[i*4+3] * 4), state[i*4+2], state[i*4+3]) for i in range(5)]
    st.sort(key = lambda s: s[0])

    now = 1
    re = 0
#     print(st)
    for i in range(5):
        ty = 0
        if(st[i][1] > pd1y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd1y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
        ty = 0
        if(st[i][1] > pd2y + HALF_PAD_HEIGHT):
            ty = 1
        elif(st[i][1] < pd2y - HALF_PAD_HEIGHT):
            ty = 2
        re += now * ty
        now *= 3
    return re
state = env.reset()

In [3]:

q_table = np.zeros([(3*3)**5, env.action_space.n])
weight_path = "q_learning_{}.npy".format(4)
if os.path.isfile(weight_path):
    with open(weight_path, 'rb') as f:
        q_table = np.load(f)

In [4]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.01
gamma = 0.99
epsilon = 0.2

# For plotting metrics
all_epochs = []
all_penalties = []
all_reward = 0

for i in range(1, 100001):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values
        next_state, reward, done, info = env.step(action) 
        next_state = encode(next_state)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1
        all_reward += reward
        state = next_state
        epochs += 1
    if i % 100 == 0:
#         clear_output(wait=True)
        print(f"Episode: {i}")
        print(f"average reward: {all_reward / 100}")
        all_reward = 0
        
print("Training finished.\n")

Episode: 100
average reward: 149.95
Episode: 200
average reward: 265.28
Episode: 300
average reward: 255.62
Episode: 400
average reward: 278.79
Episode: 500
average reward: 265.91
Episode: 600
average reward: 262.91
Episode: 700
average reward: 281.34
Episode: 800
average reward: 285.09
Episode: 900
average reward: 285.1
Episode: 1000
average reward: 308.81
Episode: 1100
average reward: 288.55
Episode: 1200
average reward: 312.98
Episode: 1300
average reward: 321.16
Episode: 1400
average reward: 317.24
Episode: 1500
average reward: 320.87
Episode: 1600
average reward: 344.39
Episode: 1700
average reward: 319.94
Episode: 1800
average reward: 367.1
Episode: 1900
average reward: 341.74
Episode: 2000
average reward: 338.4
Episode: 2100
average reward: 358.69
Episode: 2200
average reward: 376.14
Episode: 2300
average reward: 349.87
Episode: 2400
average reward: 374.4
Episode: 2500
average reward: 353.75
Episode: 2600
average reward: 393.69
Episode: 2700
average reward: 374.47
Episode: 2800


Episode: 22100
average reward: 596.24
Episode: 22200
average reward: 599.26
Episode: 22300
average reward: 570.65
Episode: 22400
average reward: 542.25
Episode: 22500
average reward: 563.71
Episode: 22600
average reward: 598.15
Episode: 22700
average reward: 570.64
Episode: 22800
average reward: 592.7
Episode: 22900
average reward: 594.62
Episode: 23000
average reward: 593.21
Episode: 23100
average reward: 590.11
Episode: 23200
average reward: 579.35
Episode: 23300
average reward: 587.04
Episode: 23400
average reward: 564.08
Episode: 23500
average reward: 588.19
Episode: 23600
average reward: 585.35
Episode: 23700
average reward: 589.51
Episode: 23800
average reward: 596.64
Episode: 23900
average reward: 565.53
Episode: 24000
average reward: 576.22
Episode: 24100
average reward: 607.37
Episode: 24200
average reward: 611.19
Episode: 24300
average reward: 588.61
Episode: 24400
average reward: 600.39
Episode: 24500
average reward: 580.16
Episode: 24600
average reward: 589.96
Episode: 2470

Episode: 43700
average reward: 601.71
Episode: 43800
average reward: 608.05
Episode: 43900
average reward: 586.85
Episode: 44000
average reward: 618.75
Episode: 44100
average reward: 646.16
Episode: 44200
average reward: 641.92
Episode: 44300
average reward: 635.93
Episode: 44400
average reward: 617.57
Episode: 44500
average reward: 597.4
Episode: 44600
average reward: 611.84
Episode: 44700
average reward: 611.6
Episode: 44800
average reward: 634.86
Episode: 44900
average reward: 616.24
Episode: 45000
average reward: 591.88
Episode: 45100
average reward: 611.57
Episode: 45200
average reward: 611.26
Episode: 45300
average reward: 625.88
Episode: 45400
average reward: 621.67
Episode: 45500
average reward: 623.85
Episode: 45600
average reward: 605.15
Episode: 45700
average reward: 606.04
Episode: 45800
average reward: 643.15
Episode: 45900
average reward: 633.09
Episode: 46000
average reward: 609.24
Episode: 46100
average reward: 644.52
Episode: 46200
average reward: 636.46
Episode: 46300

KeyboardInterrupt: 

In [ ]:
with open(weight_path, 'wb') as f:
    np.save(f, q_table)

In [6]:
import time
total_epochs, total_penalties = 0, 0
episodes = 100
total_reward = 0
for _ in range(episodes):
    state = env.reset()
    state = encode(state)
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        state = encode(state)
        if reward == -10:
            penalties += 1
        
        epochs += 1
        total_reward += reward
#         env.render()
#         time.sleep(1/60)
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
print(f"Average reward per episode: {total_reward / episodes}")

Results after 100 episodes:
Average timesteps per episode: 10801.0
Average penalties per episode: 28.63
Average reward per episode: 618.06


In [ ]:
with open("weight.coe", 'w') as f:
    f.write("memory_initialization_radix=10;\n")
    f.write("memory_initialization_vector=\n")
    for i in range(3**10):
        f.write(str(np.argmax(q_table[i])))
        if(i == 3**10 - 1):
            f.write(";\n")
        else:
            f.write(",\n")